In [217]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('scraping-github-topics-repositories')
jovian.set_colab_id('1xZ-JcAEHO-2Fe2J4ZzSQtCwJETuJiu6y')

#Top Repositories for GitHub Topics


###Pick a website and describe your objective

- Browse through different sites and pick on to scrape. Check the "Project Ideas" section for inspiration.

- Identify the information you'd like to scrape from the site. Decide the format of the output CSV file.

- Summarize your project idea and outline your strategy in a Juptyer notebook. Use the "New" button above.

####Project Outline

- We are going to scrape https://github.com/topics

- we'll get a list of topics.For each topic, we'll get topic title, topic page URL and topic description

- For each topic we'll get top 25 repositories in the topic from the topic page

- For each repository,we'll grab the repo name, username, stars and repo url.

- For each topic we'll create a csv file in the following format:

Repo Name, Username, Stars , Repo URL

####Use the requests library to download web pages

In [218]:
 !pip install requests --upgrade --quiet

In [219]:
import requests

In [220]:
topics_url ='https://github.com/topics'

In [221]:
response = requests.get(topics_url)

In [222]:
response.status_code

200

In [223]:
len(response.text)

140322

In [224]:
page_contents = response.text

In [225]:
page_contents [:1000]

'\n\n<!DOCTYPE html>\n<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark" data-a11y-animated-images="system">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n\n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-5178aee0ee76.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-217d4f9c8e70.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" media="all" rel="stylesheet" data-href="htt

In [226]:
with open('webpage.html', 'w') as f:
  f.write(page_contents)

###Use Beautiful Soup to parse and extract information

In [227]:
!pip install beautifulsoup4 --upgrade --quiet

In [228]:
from bs4 import BeautifulSoup

In [229]:
!pip install html.parse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [230]:
doc =BeautifulSoup(page_contents, 'html5lib')

In [231]:
type(doc)

bs4.BeautifulSoup

In [232]:
selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
topic_title_tags =doc.find_all('p',{'class' :selection_class})

In [233]:
len(topic_title_tags)

30

In [234]:
topic_title_tags

[<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Algorithm</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amp</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Android</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Angular</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ansible</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">API</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Arduino</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">ASP.NET</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Atom</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Awesome Lists</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amazon Web Services</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Azure</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Babel</p>,
 <p class="f3 lh-condensed m

In [235]:
topic_title_tags[:5]

[<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Algorithm</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amp</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Android</p>]

In [236]:
desc_selector ='f5 color-fg-muted mb-0 mt-1'
topic_desc_tags =doc.find_all('p', {'class' :desc_selector})

In [237]:
topic_desc_tags[:5]

[<p class="f5 color-fg-muted mb-0 mt-1">
           3D modeling is the process of virtually developing the surface and structure of a 3D object.
         </p>, <p class="f5 color-fg-muted mb-0 mt-1">
           Ajax is a technique for creating interactive web applications.
         </p>, <p class="f5 color-fg-muted mb-0 mt-1">
           Algorithms are self-contained sequences that carry out a variety of tasks.
         </p>, <p class="f5 color-fg-muted mb-0 mt-1">
           Amp is a non-blocking concurrency library for PHP.
         </p>, <p class="f5 color-fg-muted mb-0 mt-1">
           Android is an operating system built by Google designed for mobile devices.
         </p>]

In [238]:
topic_title_tag0 =  topic_title_tags[0]

In [239]:
topic_title_tag0

<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>

In [240]:
topic_title_tag0.parent

<a class="no-underline flex-1 d-flex flex-column" href="/topics/3d">
        <p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>
        <p class="f5 color-fg-muted mb-0 mt-1">
          3D modeling is the process of virtually developing the surface and structure of a 3D object.
        </p>
      </a>

In [241]:
topic_link_tags =doc.find_all('a', {'class':'no-underline flex-1 d-flex flex-column'})

In [242]:
len(topic_link_tags)

30

In [243]:
topic_link_tags[0]

<a class="no-underline flex-1 d-flex flex-column" href="/topics/3d">
        <p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>
        <p class="f5 color-fg-muted mb-0 mt-1">
          3D modeling is the process of virtually developing the surface and structure of a 3D object.
        </p>
      </a>

In [244]:
topic_link_tags[0]['href']

'/topics/3d'

In [245]:
topic0_url= "https://github.com"+topic_link_tags[0]['href']
print(topic0_url)

https://github.com/topics/3d


In [246]:
topic_title_tags[:5]

[<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Algorithm</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amp</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Android</p>]

In [247]:
topic_title_tags[0].text

'3D'

In [248]:
topic_titles = []
for tag in topic_title_tags :
  topic_titles.append(tag.text)
print (topic_titles)

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android', 'Angular', 'Ansible', 'API', 'Arduino', 'ASP.NET', 'Atom', 'Awesome Lists', 'Amazon Web Services', 'Azure', 'Babel', 'Bash', 'Bitcoin', 'Bootstrap', 'Bot', 'C', 'Chrome', 'Chrome extension', 'Command line interface', 'Clojure', 'Code quality', 'Code review', 'Compiler', 'Continuous integration', 'COVID-19', 'C++']


In [249]:
topic_descs = []
for tag in topic_desc_tags:
  topic_descs.append(tag.text.strip())
topic_descs[:5]

['3D modeling is the process of virtually developing the surface and structure of a 3D object.',
 'Ajax is a technique for creating interactive web applications.',
 'Algorithms are self-contained sequences that carry out a variety of tasks.',
 'Amp is a non-blocking concurrency library for PHP.',
 'Android is an operating system built by Google designed for mobile devices.']

In [250]:
topic_urls = []
base_url ='https://github.com'
for tag in topic_link_tags:
  topic_urls.append(base_url +tag['href'])
topic_urls

['https://github.com/topics/3d',
 'https://github.com/topics/ajax',
 'https://github.com/topics/algorithm',
 'https://github.com/topics/amphp',
 'https://github.com/topics/android',
 'https://github.com/topics/angular',
 'https://github.com/topics/ansible',
 'https://github.com/topics/api',
 'https://github.com/topics/arduino',
 'https://github.com/topics/aspnet',
 'https://github.com/topics/atom',
 'https://github.com/topics/awesome',
 'https://github.com/topics/aws',
 'https://github.com/topics/azure',
 'https://github.com/topics/babel',
 'https://github.com/topics/bash',
 'https://github.com/topics/bitcoin',
 'https://github.com/topics/bootstrap',
 'https://github.com/topics/bot',
 'https://github.com/topics/c',
 'https://github.com/topics/chrome',
 'https://github.com/topics/chrome-extension',
 'https://github.com/topics/cli',
 'https://github.com/topics/clojure',
 'https://github.com/topics/code-quality',
 'https://github.com/topics/code-review',
 'https://github.com/topics/compil

In [251]:
!pip install pandas --quiet

In [252]:
import pandas as pd

In [253]:
topics_dict ={
    'title' :topic_titles,
    'description' :topic_descs,
    'url' :topic_urls
}

To check the dictionary

In [254]:
topics_dict

{'title': ['3D',
  'Ajax',
  'Algorithm',
  'Amp',
  'Android',
  'Angular',
  'Ansible',
  'API',
  'Arduino',
  'ASP.NET',
  'Atom',
  'Awesome Lists',
  'Amazon Web Services',
  'Azure',
  'Babel',
  'Bash',
  'Bitcoin',
  'Bootstrap',
  'Bot',
  'C',
  'Chrome',
  'Chrome extension',
  'Command line interface',
  'Clojure',
  'Code quality',
  'Code review',
  'Compiler',
  'Continuous integration',
  'COVID-19',
  'C++'],
 'description': ['3D modeling is the process of virtually developing the surface and structure of a 3D object.',
  'Ajax is a technique for creating interactive web applications.',
  'Algorithms are self-contained sequences that carry out a variety of tasks.',
  'Amp is a non-blocking concurrency library for PHP.',
  'Android is an operating system built by Google designed for mobile devices.',
  'Angular is an open source web application platform.',
  'Ansible is a simple and powerful automation engine.',
  'An API (Application Programming Interface) is a collec

In [255]:
topics_df =pd.DataFrame(topics_dict)

In [256]:
topics_df

,title,description,url
0,3D,3D modeling is the process of virtually develo...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android
5,Angular,Angular is an open source web application plat...,https://github.com/topics/angular
6,Ansible,Ansible is a simple and powerful automation en...,https://github.com/topics/ansible
7,API,An API (Application Programming Interface) is ...,https://github.com/topics/api
8,Arduino,Arduino is an open source hardware and softwar...,https://github.com/topics/arduino
9,ASP.NET,ASP.NET is a web framework for building modern...,https://github.com/topics/aspnet


create a CSV file with extracted information

In [257]:
topics_df.to_csv('topics.csv', index=None)

Getting Information out of a topic page

In [258]:
topic_page_url = topic_urls[0]

In [259]:
topic_page_url

'https://github.com/topics/3d'

In [260]:
response = requests.get(topic_page_url)

In [261]:
response.status_code

200

In [262]:
len(response.text)

431094

In [263]:
topic_doc = BeautifulSoup(response.text ,'html5lib')

In [264]:
h3_selection_class ='f3 color-fg-muted text-normal lh-condensed'
repo_tags=topic_doc.find_all('h3',{'class' :h3_selection_class})

In [265]:
len(repo_tags)

20

In [266]:
repo_tags[0]

<h3 class="f3 color-fg-muted text-normal lh-condensed">
          <a data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
            mrdoob
</a>          /
          <a class="text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":576201,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="517d3d5cb9d89752156923904a4238816bc9b51ab7772f3e3644ce897d8dd4e5" data-turbo="false" data-view-component="true" href="/mrdoo

In [267]:
a_tags = repo_tags[0].find_all('a')

In [268]:
a_tags

[<a data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
             mrdoob
 </a>,
 <a class="text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":576201,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="517d3d5cb9d89752156923904a4238816bc9b51ab7772f3e3644ce897d8dd4e5" data-turbo="false" data-view-component="true" href="/mrdoob/three.js">
             three.js
 </a>]

In [269]:
a_tags[0]

<a data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
            mrdoob
</a>

In [270]:
a_tags[0].text.strip()

'mrdoob'

In [271]:
a_tags[1].text.strip()

'three.js'

In [272]:
base_url='https://github.com'
repo_url=base_url+a_tags[1]['href']
print(repo_url)

https://github.com/mrdoob/three.js


getting the stars info

In [273]:
star_tags=topic_doc.find_all('span',{'class':'Counter js-social-count'})

In [274]:
len(star_tags)

20

In [275]:
star_tags[0]

<span aria-label="84739 users starred this repository" class="Counter js-social-count" data-pjax-replace="true" data-plural-suffix="users starred this repository" data-singular-suffix="user starred this repository" data-turbo-replace="true" data-view-component="true" id="repo-stars-counter-star" title="84,739">84.7k</span>

In [276]:
star_tags[0].text.strip()

'84.7k'

In [277]:
def parse_star_count(stars_str):
   stars_str=stars_str.strip()
   if stars_str[-1] =='k':
     return int(float(stars_str[:-1])*1000)
     return int(stars_str)

In [278]:
parse_star_count(star_tags[0].text.strip())

84700

In [279]:
def get_repo_info(h3_tag, star_tag):
  #returns all the required info about a repository
  a_tags=h3_tag.find_all('a')
  username= a_tags[0].text.strip()
  repo_name= a_tags[1].text.strip()
  repo_url = base_url+a_tags[1]['href']
  stars= parse_star_count(star_tag.text.strip())
  return username, repo_name, stars,repo_url

In [280]:
get_repo_info(repo_tags[0], star_tags[0])

('mrdoob', 'three.js', 84700, 'https://github.com/mrdoob/three.js')

In [281]:
topic_repos_dict = {
    'username' :[] ,
    'repo_name' :[],
    'stars' : [],
    'repo_url' :[]    
    }

for i in range(len(repo_tags)):
  repo_info =get_repo_info(repo_tags[i], star_tags[i])
  topic_repos_dict['username'].append(repo_info[0])
  topic_repos_dict['repo_name'].append(repo_info[1])
  topic_repos_dict['stars'].append(repo_info[2])
  topic_repos_dict['repo_url'].append(repo_info[3])

In [282]:
topic_repos_dict

{'username': ['mrdoob',
  'libgdx',
  'pmndrs',
  'BabylonJS',
  'ssloy',
  'aframevr',
  'lettier',
  'FreeCAD',
  'metafizzy',
  'CesiumGS',
  'timzhang642',
  'isl-org',
  'a1studmuffin',
  'blender',
  'domlysz',
  'spritejs',
  'openscad',
  'tensorspace-team',
  'jagenjo',
  'YadiraF'],
 'repo_name': ['three.js',
  'libgdx',
  'react-three-fiber',
  'Babylon.js',
  'tinyrenderer',
  'aframe',
  '3d-game-shaders-for-beginners',
  'FreeCAD',
  'zdog',
  'cesium',
  '3D-Machine-Learning',
  'Open3D',
  'SpaceshipGenerator',
  'blender',
  'BlenderGIS',
  'spritejs',
  'openscad',
  'tensorspace',
  'webglstudio.js',
  'PRNet'],
 'stars': [84700,
  20400,
  19300,
  18200,
  14500,
  14500,
  13600,
  12000,
  9300,
  9100,
  8300,
  7300,
  7200,
  6400,
  5500,
  5000,
  5000,
  4700,
  4700,
  4700],
 'repo_url': ['https://github.com/mrdoob/three.js',
  'https://github.com/libgdx/libgdx',
  'https://github.com/pmndrs/react-three-fiber',
  'https://github.com/BabylonJS/Babylon.js',

Final Code

In [283]:
import os

def get_topic_page(topic_url):
    #download the page
  response = requests.get(topic_url)
  #check successful response
  if response.status_code !=200:
    raise Exception('Failed to load page{}'.format(topic_url))
  #Parse using Beautiful Soup
  topic_doc = BeautifulSoup(response.text, 'html.parser')
  return topic_doc

def get_repo_info(h3_tag, star_tag):
  #returns all the required info about a repository
  a_tags=h3_tag.find_all('a')
  username= a_tags[0].text.strip()
  repo_name= a_tags[1].text.strip()
  repo_url = base_url+a_tags[1]['href']
  stars= parse_star_count(star_tag.text.strip())
  return username, repo_name, stars,repo_url

def get_topic_repos(topic_doc):
  #Get the h3 tags containing repo title,Url and username
  h3_selection_class ='f3 color-fg-muted text-normal lh-condensed'
  repo_tags=topic_doc.find_all('h3',{'class' :h3_selection_class})
  #get star tags
  star_tags=topic_doc.find_all('span',{'class':'Counter js-social-count'})

  topic_repos_dict = {
    'username' :[] ,
    'repo_name' :[],
    'stars' : [],
    'repo_url' :[]    
  }  
 
  #get repo info
  for i in range(len(repo_tags)):
   repo_info =get_repo_info(repo_tags[i], star_tags[i])
   topic_repos_dict['username'].append(repo_info[0])
   topic_repos_dict['repo_name'].append(repo_info[1])
   topic_repos_dict['stars'].append(repo_info[2])
   topic_repos_dict['repo_url'].append(repo_info[3])

  return pd.DataFrame(topic_repos_dict)

def scrape_topic(topic_url, path):
  
  if os.path.exists(path):
    print("The file {} already exists.Skipping...".format(path))
    return
  topic_df=get_topic_repos(get_topic_page(topic_url))
  topic_df.to_csv(path, index=None)

Write a single function to:

1. Get the list of topics from topics page

2. Get the list of top repos from the individual topic repos

3. For each topic, create a CSV of the top repos for the topic

In [284]:
topics_url

'https://github.com/topics'

In [285]:
def get_topic_titles(doc):
    selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
    topic_title_tags =doc.find_all('p',{'class' :selection_class})
    topic_titles = []
    for tag in topic_title_tags :
        topic_titles.append(tag.text)
    return topic_titles
def get_topic_descs(doc):
    desc_selector ='f5 color-fg-muted mb-0 mt-1'
    topic_desc_tags =doc.find_all('p', {'class' :desc_selector})
    topic_descs = []
    for tag in topic_desc_tags:
        topic_descs.append(tag.text.strip())
    return topic_descs
def get_topic_urls(doc):
    topic_link_tags =doc.find_all('a', {'class':'no-underline flex-1 d-flex flex-column'})
    topic_urls = []
    base_url ='https://github.com'
    for tag in topic_link_tags:
        topic_urls.append(base_url +tag['href'])
    return topic_urls
def scrape_topics():
   topics_url = 'https://github.com/topics'
   response = requests.get(topics_url)
   if response.status_code !=200:
    raise Exception('Failed to load page{}'.format(topic_url))
   topics_dict = {
       'title':get_topic_titles(doc),
       'description': get_topic_descs(doc),
       'url': get_topic_urls(doc)
   }
   return pd.DataFrame(topics_dict)

In [286]:
def scrape_topics_repos():
  print('Scraping list of topics')
  topics_df = scrape_topics()
  
  os.makedirs('data', exist_ok=True)

  for index, row in topics_df.iterrows():
    print('scraping top repositories for "{}"'.format(row['title']))
    scrape_topic (row['url'], 'data/{}.csv'.format(row['title']))

In [287]:
scrape_topics_repos()

Scraping list of topics
scraping top repositories for "3D"
The file data/3D.csv already exists.Skipping...
scraping top repositories for "Ajax"
The file data/Ajax.csv already exists.Skipping...
scraping top repositories for "Algorithm"
The file data/Algorithm.csv already exists.Skipping...
scraping top repositories for "Amp"
The file data/Amp.csv already exists.Skipping...
scraping top repositories for "Android"
The file data/Android.csv already exists.Skipping...
scraping top repositories for "Angular"
The file data/Angular.csv already exists.Skipping...
scraping top repositories for "Ansible"
The file data/Ansible.csv already exists.Skipping...
scraping top repositories for "API"
The file data/API.csv already exists.Skipping...
scraping top repositories for "Arduino"
The file data/Arduino.csv already exists.Skipping...
scraping top repositories for "ASP.NET"
The file data/ASP.NET.csv already exists.Skipping...
scraping top repositories for "Atom"
The file data/Atom.csv already exists

In [288]:
import jovian

In [293]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...


[jovian] Error: Looks like the notebook is missing output cells, please save the notebook and try jovian.commit again.


Committed successfully! https://jovian.ai/navyagudise2000/scraping-github-topics-repositories


'https://jovian.ai/navyagudise2000/scraping-github-topics-repositories'